In [1]:
from re import L
import tensorflow as tf 
from tensorflow.keras.layers import Conv2D,UpSampling2D,MaxPooling2D,Input
from tensorflow.keras.models import Model 
import numpy as np 
from matplotlib import pyplot as plt 
from os import listdir
from PIL import Image,ImageOps
import random

In [21]:
#dataset loading and pre processing data 
def data_loader(dataset_path):
    dataset_color = []
    #shuffling pictures
    dir = listdir(dataset_path)
    for fruit_type in dir:
        file = listdir(dataset_path+'/'+fruit_type)
        #random.shuffle(file)
        for filename in file:
            #print(dataset_path+'/'+fruit_type+'/'+filename)
            fruit = Image.open(dataset_path+'/'+fruit_type+'/'+filename)
            fruit_color  = np.asarray(fruit)
            dataset_color.append(fruit_color)
    #return the dataset 
    return np.array(dataset_color)




In [22]:
data = data_loader('dataset')

In [42]:
input_layer = Input((100,100,3))
layer = Conv2D(50,(3,3),activation = 'relu',padding ='same')(input_layer)
layer = MaxPooling2D(pool_size = (4,4),strides =(4,4))(layer)
layer = Conv2D(50,(3,3),activation = 'sigmoid',padding= 'same')(layer)
layer = MaxPooling2D(pool_size = (4,4),strides =(6,6))(layer)
layer = Conv2D(1,(3,3),activation = 'sigmoid')(layer)
layer = Conv2D(1,(2,2),activation = 'sigmoid')(layer)
layer = UpSampling2D((5,5))(layer)
layer = Conv2D(50,(3,3),activation = 'sigmoid',padding = 'same')(layer)
layer = UpSampling2D((5,5))(layer)
layer = Conv2D(50,(3,3),activation = 'sigmoid',padding = 'same')(layer)
layer = UpSampling2D((4,4))(layer)
layer = Conv2D(3,(3,3),activation = 'linear',padding = 'same')(layer)
model = Model(input_layer,layer)
model.summary()


Model: "functional_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 100, 100, 50)      1400      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 25, 25, 50)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 25, 25, 50)        22550     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 4, 4, 50)          0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 2, 2, 1)           451       
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 1, 1, 1)         

In [43]:
model.compile(optimizer = 'rmsprop',loss = 'mse')
model.fit(data,data,batch_size = 50,epochs = 4)

Epoch 1/4
33/33 [==============================] - 31s 946ms/step - loss: 24186.9082
Epoch 2/4
33/33 [==============================] - 32s 971ms/step - loss: 20730.3730
Epoch 3/4
33/33 [==============================] - 33s 1s/step - loss: 17992.2754
Epoch 4/4
33/33 [==============================] - 33s 987ms/step - loss: 15770.5527


In [44]:
test = model.predict(data[0:3])

In [45]:
image = Image.fromarray(np.array(test[0]).astype(np.uint8))
image.show()